In [124]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

from joblib import dump, load

In [125]:
# dipole = pd.read_csv("dipole_moments.csv")
# shielding = pd.read_csv("magnetic_shielding_tensors.csv")
# charge = pd.read_csv("mulliken_charges.csv")
# PE = pd.read_csv("potential_energy.csv")
# contribution = pd.read_csv("scalar_coupling_contributions.csv")
# structures = pd.read_csv("structures.csv")

test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")


In [126]:
dipole.head()

,molecule_name,X,Y,Z
0,dsgdb9nsd_000001,0.0000,0.0,0.0000
1,dsgdb9nsd_000002,-0.0002,0.0,1.6256
2,dsgdb9nsd_000003,0.0000,0.0,-1.8511
3,dsgdb9nsd_000005,0.0000,0.0,-2.8937
4,dsgdb9nsd_000007,0.0000,0.0,0.0000


In [127]:
shielding.head()

,molecule_name,atom_index,XX,YX,ZX,XY,YY,ZY,XZ,YZ,ZZ
0,dsgdb9nsd_000001,0,195.3150,0.0000,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170
1,dsgdb9nsd_000001,1,31.3410,-1.2317,4.0544,-1.2317,28.9546,-1.7173,4.0546,-1.7173,34.0861
2,dsgdb9nsd_000001,2,31.5814,1.2173,-4.1474,1.2173,28.9036,-1.6036,-4.1476,-1.6036,33.8967
3,dsgdb9nsd_000001,3,31.5172,4.1086,1.2723,4.1088,33.9068,1.6950,1.2724,1.6951,28.9579
4,dsgdb9nsd_000001,4,31.4029,-4.0942,-1.1793,-4.0944,34.0776,1.6259,-1.1795,1.6260,28.9013


In [128]:
charge.head()

,molecule_name,atom_index,mulliken_charge
0,dsgdb9nsd_000001,0,-0.535689
1,dsgdb9nsd_000001,1,0.133921
2,dsgdb9nsd_000001,2,0.133922
3,dsgdb9nsd_000001,3,0.133923
4,dsgdb9nsd_000001,4,0.133923


In [129]:
PE.head()

,molecule_name,potential_energy
0,dsgdb9nsd_000001,-40.52368
1,dsgdb9nsd_000002,-56.56025
2,dsgdb9nsd_000003,-76.42608
3,dsgdb9nsd_000005,-93.42849
4,dsgdb9nsd_000007,-79.83869


In [130]:
contribution.head()

,molecule_name,atom_index_0,atom_index_1,type,fc,sd,pso,dso
0,dsgdb9nsd_000001,1,0,1JHC,83.0224,0.254579,1.25862,0.272010
1,dsgdb9nsd_000001,1,2,2JHH,-11.0347,0.352978,2.85839,-3.433600
2,dsgdb9nsd_000001,1,3,2JHH,-11.0325,0.352944,2.85852,-3.433870
3,dsgdb9nsd_000001,1,4,2JHH,-11.0319,0.352934,2.85855,-3.433930
4,dsgdb9nsd_000001,2,0,1JHC,83.0222,0.254585,1.25861,0.272013


In [131]:
train.type.unique()

array(['1JHC', '2JHH', '1JHN', '2JHN', '2JHC', '3JHH', '3JHC', '3JHN'],
      dtype=object)

In [132]:
structures.head(20)

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397
5,dsgdb9nsd_000002,0,N,-0.040426,1.024108,0.062564
6,dsgdb9nsd_000002,1,H,0.017257,0.012545,-0.027377
7,dsgdb9nsd_000002,2,H,0.915789,1.358745,-0.028758
8,dsgdb9nsd_000002,3,H,-0.520278,1.343532,-0.775543
9,dsgdb9nsd_000003,0,O,-0.034360,0.977540,0.007602


In [133]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [134]:
print(train.shape)
print(test.shape)

(4658147, 6)
(2505542, 5)


In [135]:
type_2jhh = contribution[contribution.type == "2JHH"]

In [136]:
#sns.pairplot(type_2jhh)

In [137]:
# g = sns.FacetGrid(train,  hue="type",height=4, aspect=2)
# g = (g.map(sns.kdeplot, "scalar_coupling_constant").add_legend())
# g.fig.suptitle('Scalar Coupling Constant Distribution by Type')
# g.axes[0,0].set_xlabel("Scalar Coupling Constant(Hz)")


In [138]:
names = structures.groupby(by='molecule_name')['atom'].agg(lambda x :''.join(x))

In [139]:
train["names"] = train["molecule_name"].apply(lambda x: names.loc[x])

In [140]:
names[names == "CCCCCCHHHHHHHHHHHHOOOOOO"]

Series([], Name: atom, dtype: object)

In [141]:
benzene = names[names == "CCCCCCHHHHHH"].index

In [142]:
#train[train["molecule_name"] == "dsgdb9nsd_000719"].shape
#len(benzene)*45

In [143]:
#benzene__000719 = structures[structures["molecule_name"] == "dsgdb9nsd_000719"]

In [144]:
def map_atom_info(df, atom_idx):
    df = pd.merge(df, structures, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

train = map_atom_info(train, 0)
train = map_atom_info(train, 1)

test = map_atom_info(test, 0)
test = map_atom_info(test, 1)


In [145]:
train_p_0 = train[['x_0', 'y_0', 'z_0']].values
train_p_1 = train[['x_1', 'y_1', 'z_1']].values
test_p_0 = test[['x_0', 'y_0', 'z_0']].values
test_p_1 = test[['x_1', 'y_1', 'z_1']].values

train['dist_speedup'] = np.linalg.norm(train_p_0 - train_p_1, axis=1)
test['dist_speedup'] = np.linalg.norm(test_p_0 - test_p_1, axis=1)

In [146]:
# g = sns.FacetGrid(train,  hue="type",height=4, aspect=2)
# g = (g.map(sns.kdeplot, "dist_speedup").add_legend())
# g.fig.suptitle('Distance distribution by Type')
# g.axes[0,0].set_xlabel("Distance")

In [147]:
# rs = np.random.RandomState(33)
# d = pd.DataFrame(data=train,
#                  columns=train.columns)

# # Compute the correlation matrix
# corr = d.corr()

# # Generate a mask for the upper triangle
# mask = np.zeros_like(corr, dtype=np.bool)
# mask[np.triu_indices_from(mask)] = True

# # Set up the matplotlib figure
# f, ax = plt.subplots(figsize=(11, 9))

# # Generate a custom diverging colormap
# cmap = sns.diverging_palette(220, 10, as_cmap=True)

# # Draw the heatmap with the mask and correct aspect ratio
# sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
#             square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [148]:
#train.corr()

In [149]:
#sns.kdeplot(structures.groupby("molecule_name").count()["atom"])

In [150]:
train = pd.concat([train, pd.get_dummies(train['type'])], axis=1)
test = pd.concat([test, pd.get_dummies(test['type'])], axis=1)
train = pd.concat([train, pd.get_dummies(train['type'])], axis=1)
test = pd.concat([test, pd.get_dummies(test['type'])], axis=1)
train = pd.concat([train, pd.get_dummies(train['atom_1'])], axis=1)
test = pd.concat([test, pd.get_dummies(test['atom_1'])], axis=1)
test.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'type', 'atom_0',
       'x_0', 'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1', 'dist_speedup',
       '1JHC', '1JHN', '2JHC', '2JHH', '2JHN', '3JHC', '3JHH', '3JHN', '1JHC',
       '1JHN', '2JHC', '2JHH', '2JHN', '3JHC', '3JHH', '3JHN', 'C', 'H', 'N'],
      dtype='object')

In [151]:
X = train[["atom_index_0", "atom_index_1", 'x_0', 'y_0', 'z_0',
       'x_1', 'y_1', 'z_1', 'dist_speedup', '1JHC', '1JHN', '2JHC', '2JHH',
       '2JHN', '3JHC', '3JHH', '3JHN', "C", "H", "N"]]
y = train["scalar_coupling_constant"]
final_X = train[["atom_index_0", "atom_index_1", 'x_0', 'y_0', 'z_0',
       'x_1', 'y_1', 'z_1', 'dist_speedup', '1JHC', '1JHN', '2JHC', '2JHH',
       '2JHN', '3JHC', '3JHH', '3JHN', "C", "H", "N"]]

In [55]:

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size = 0.3, random_state=4)

# max_depth = 30

# regr_rf = RandomForestRegressor(n_estimators=100, max_depth=max_depth,
#                                 random_state=2)

# regr_rf.fit(X_train, y_train)

# # Predict on new data
# y_rf = regr_rf.predict(X_test)

# dump(regr_rf, 'rf_1.joblib') 

In [76]:
load('rf_1.joblib')

['rf_1.joblib']

In [71]:
y_rf[:10]

array([  0.97831856,  83.87923147,   7.67467609,   1.51041702,
         3.43932925,  -1.79636819,   4.64114649, -11.57651   ,
         3.45036242,   6.71922875])

In [58]:
y.head()

0    84.8076
1   -11.2570
2   -11.2548
3   -11.2543
4    84.8074
Name: scalar_coupling_constant, dtype: float64

In [113]:
len(regr_rf.feature_importances_)

28

In [65]:
y_train.shape

(3260702,)

In [62]:
len(y_rf)

1397445

In [66]:
np.mean(np.log(y_rf-y_test))

C:\Users\Sean\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.


-0.20673908201491067

In [119]:
X_train.columns

Index(['atom_index_0', 'atom_index_1', 'x_0', 'y_0', 'z_0', 'x_1', 'y_1',
       'z_1', 'dist_speedup', '1JHC', '1JHC', '1JHN', '1JHN', '2JHC', '2JHC',
       '2JHH', '2JHH', '2JHN', '2JHN', '3JHC', '3JHC', '3JHH', '3JHH', '3JHN',
       '3JHN', 'C', 'H', 'N'],
      dtype='object')

In [114]:
final_X.columns


Index(['atom_index_0', 'atom_index_1', 'x_0', 'y_0', 'z_0', 'x_1', 'y_1',
       'z_1', 'dist_speedup', '1JHC', '1JHN', '2JHC', '2JHH', '2JHN', '3JHC',
       '3JHH', '3JHN', 'C', 'H', 'N'],
      dtype='object')

In [156]:
test_predict = regr_rf.predict(final_X)

MemoryError: 